In [5]:


import pandas as pd
import numpy as np
df = pd.read_csv('input/train.csv')
size = df.loc[df['track_id'].idxmax()]['track_id']
playlistsForTrack = np.empty([size], dtype=set)
for i in range(size):
    playlistsForTrack[i] = set(df[df['track_id']==i]['playlist_id'])
bestSimilarTracks = sparse.csr_matrix((size, size), dtype=np.double)
for i in range(size):
    temp = np.zeros((size), dtype=np.int8)
    bestSimilarTracks[i][i] = [0,i]
    for j in range(i):
        den = ((len(playlistsForTrack[i])*1.2)*(len(playlistsForTrack[j]))*0.8)**0.5+5
        similarity = len(playlistsForTrack[i] & playlistsForTrack[j])/den
        bestSimilarTracks[i][j] = [-1*similarity, j]
    for j in range(i+1, size):
        den = ((len(playlistsForTrack[i])*1.2)*(len(playlistsForTrack[j]))*0.8)**0.5+5
        similarity = len(playlistsForTrack[i] & playlistsForTrack[j])/den
        bestSimilarTracks[i][j] = [-1*similarity, j]
    #bestSimilarTracks[i] = bestSimilarTracks[i][bestSimilarTracks[i][:,0].argpartition(10)][0:10]
    bestSimilarTracks[i] = bestSimilarTracks[i][bestSimilarTracks[i][:,0] !=0]#[:,1]
    bestSimilarTracks[i][:,0] = -1*bestSimilarTracks[i][:,0]


In [30]:
from scipy.sparse import csr_matrix, vstack
def buildURM():
    df = pd.read_csv('input/train.csv')
    sizeTracks = df.loc[df['track_id'].idxmax()]['track_id']
    sizePlaylists = df.loc[df['playlist_id'].idxmax()]['playlist_id']
    URM = csr_matrix((sizeTracks, sizePlaylists), dtype=np.int8)
    print(URM)
    for i in range(1):
        temp = np.zeros((sizePlaylists), dtype=np.int8)
        playlistsForTrack = df[df['track_id']==i]['playlist_id']
        for playlist in playlistsForTrack:
            temp[playlist] = 1
            print(playlist)
        temp = csr_matrix(temp)
        URM = vstack([URM, temp])
    return URM
URM = buildURM()


25
7550
9219
9586
11660
12213
15874
18259
20283
23122
24183
25661
27662
28160
31702
33059
34400
36546
37029
37400
37668
38356
39839
41319
42481
48513


In [24]:
row_start = URM.indptr[0]
row_end = URM.indptr[1]
print(URM.indices[row_start:row_end])

[]


[array([[0.05143436328800134, 67],
       [0.008053270688703434, 80],
       [0.020843232453405268, 89],
       [0.021505432064939194, 96]], dtype=object)
 array([[0.1038421748852724, 15],
       [0.03779258897522805, 21],
       [0.008054315777311126, 37],
       [0.0158421792009815, 42],
       [0.0194500343098725, 88]], dtype=object)
 array([[0.02122248785617707, 19],
       [0.009968343088755233, 32],
       [0.014006547673450392, 46],
       [0.010548547988876052, 75],
       [0.02769503474387674, 76]], dtype=object)
 ... None None None]


In [ ]:
from scipy.sparse import coo_matrix, vstack
>>> A = coo_matrix([[1, 2], [3, 4]])

In [ ]:
def merge(a, b):
    contentSimilarity = pd.read_csv('similarity/content.csv')
    cbfSimilarity = pd.read_csv('similarity/cbf.csv')
    hybridMatrix = a*contentSimilarity + b*cbfSimilarity
    return hybridMatrix
bestSimilarTracks = merge(0.3, 0.7)
df1 = pd.read_csv('input/target_playlists.csv')
playlists = np.array(df1['playlist_id'])
import random
submission = np.empty((len(playlists),2), dtype=object)
cont = -1
for playlist in playlists:
    cont = cont + 1
    tracks = np.array(df[df['playlist_id']==playlist]['track_id'])
    submission[cont][0] = playlist
    tracksSet = set(tracks)
    best = {}
    temp = 1
    for track in tracks:
        if cont < 5000:
            temp -= 1/(len(tracks)+1)
        row_start = bestSimilarTracks.indptr[track]
        row_end = bestSimilarTracks.indptr[track+1]
        similarTracks = bestSimilarTracks.indices[row_start:row_end]
        similarityValues = bestSimilarTracks.data[row_start:row_end]
        for i in range(0, len(similarTracks)):
            if not similarTracks[i] in tracksSet:
                if similarTracks[i] in best:
                    best[similarTracks[i]]=best[similarTracks[i]]-similarityValues[i]*temp
                else:
                    best[similarTracks[i]]=-1*similarityValues[i]*temp
    preSorted = [[v, k] for k,v in best.items()]
    best = np.empty((max(11,len(preSorted)), 2), dtype=object)
    for i in range(len(preSorted)):
        best[i] = preSorted[i]
    if len(preSorted) < 11:
        for i in range(len(preSorted), 11):
            best[i] = [0, random.randint(0, 20000)]
        print(best)
    best = best[best[:,0].argpartition(10)][0:10]
    best = best[best[:,0].argsort()][:,1]
    best = str(best)
    submission[cont][1] = best[1:len(best)-1]
df2 = pd.DataFrame(submission, columns=['playlist_id','track_ids'])
df2.to_csv('something.csv', index=False)



In [1]:
import pandas as pd
df1 = pd.read_csv('input/target_playlists.csv')

In [5]:
import scipy.sparse as sps
import numpy as np
c = np.empty(0)

In [153]:
import numpy as np
import pandas as pd
from scipy import sparse as sps
from sklearn.preprocessing import MultiLabelBinarizer, normalize
from sklearn import feature_extraction


class Builder(object):
    
    def __init__(self):
        self.train= pd.read_csv('input/train.csv')
        self.target_playlists = pd.read_csv('input/target_playlists.csv')
        self.tracks = pd.read_csv('input/tracks.csv')
        self.playlists = self.get_playlists()
        self.tracks_inside_playlists_train = np.empty((len(self.playlists)), dtype=object)

    def get_train_pd(self):
        return self.train

    def get_target_playlists_pd(self):
        return self.target_playlists

    def get_tracks_pd(self):
        return self.tracks

    def get_ordered_target_playlists(self):
        return np.array(self.target_playlists['playlist_id'])[0:5000]

    def get_unordered_target_playlists(self):
        return np.array(self.target_playlists['playlist_id'])[5000:]

    def get_tracks_inside_playlist_train(self, playlist):
        return self.tracks_inside_playlists_train[playlist]
    
    
    def train_test_holdout(self, train_perc):
        playlistsSize = len(self.get_playlists())
        tracksSize = len(self.get_tracks())
        target_playlists = self.get_target_playlists()
        cont = 0
        URM_test_row = np.empty(0)
        URM_test_col = np.empty(0)
        URM_test_values = np.empty(0)
        URM_train_row = np.empty(0)
        URM_train_col = np.empty(0)
        URM_train_values = np.empty(0)
        for playlist in range(0,playlistsSize):
            tracks = np.array(self.train[self.train['playlist_id']==playlist]['track_id'])
            if cont < len(target_playlists) and playlist == target_playlists[cont]:
                train = tracks[0:int(len(tracks)*train_perc)]
                test = tracks[int(len(tracks)*train_perc):]
                URM_train_row = np.append(URM_train_row, [playlist]*len(train))
                URM_train_col = np.append(URM_train_col, train)
                URM_train_values = np.append(URM_train_values, [1]*len(train))
                URM_test_row = np.append(URM_test_row, [playlist]*len(test))
                URM_test_col = np.append(URM_test_col, test)
                URM_test_values = np.append(URM_test_values, [1]*len(test))
                cont = cont + 1
                self.tracks_inside_playlists_train[playlist] = train
            else:
                URM_train_row = np.append(URM_train_row, [playlist]*len(tracks))
                URM_train_col = np.append(URM_train_col, tracks)
                URM_train_values = np.append(URM_train_values, [1]*len(tracks))
            


        self.URM_train = sps.csr_matrix( (URM_train_values,(URM_train_row, URM_train_col)), shape=(playlistsSize, tracksSize))
        self.URM_test = sps.csr_matrix( (URM_test_values,(URM_test_row, URM_test_col)), shape=(playlistsSize, tracksSize))

        return self.URM_train, self.URM_test
    
    def get_tracks(self):
        tracks = self.tracks['track_id'].unique()
        return np.sort(tracks)
    
    def get_playlists(self):
        playlists = self.train['playlist_id'].unique()
        return np.sort(playlists)
    
    def get_target_playlists(self):
        target_playlists = self.target_playlists['playlist_id'].unique()
        return np.sort(target_playlists)
    
    def get_artists(self):
        artists = self.tracks['artist_id'].unique()
        return np.sort(artists)
    
    def get_albums(self):
        albums = self.tracks['album_id'].unique()
        return np.sort(albums)
    
    def get_durations(self):
        durations = self.tracks['duration_sec'].unique()
        return np.sort(durations)
    
    def get_target_playlist_index(self, target_playlist):
        return np.where(self.playlists == target_playlist)[0][0] #DA RIVEDERE
    
    def get_URM(self):
        grouped = self.train.groupby('playlist_id', as_index=True).apply(lambda x: list(x['track_id']))
        self.URM = MultiLabelBinarizer(classes=self.get_tracks(), sparse_output=True).fit_transform(grouped)
        return self.URM
    
    def get_URM_transpose(self):
        grouped = self.train.groupby('track_id', as_index=True).apply(lambda x: list(x['playlist_id']))
        self.URM = MultiLabelBinarizer(classes=self.get_playlists(), sparse_output=True).fit_transform(grouped)
        return self.URM
    
    def get_ICM(self):
        artists = self.tracks.reindex(columns=['track_id', 'artist_id'])
        artists.sort_values(by='track_id', inplace=True)
        artists_list = [[a] for a in artists['artist_id']]
        icm_artists = MultiLabelBinarizer(classes=self.get_artists(), sparse_output=True).fit_transform(artists_list)
        icm_artists_csr = icm_artists.tocsr()
        #return icm_artists_csr
        
        albums = self.tracks.reindex(columns=['track_id', 'album_id'])
        albums.sort_values(by='track_id', inplace=True)
        albums_list = [[a] for a in albums['album_id']]
        icm_albums = MultiLabelBinarizer(classes=self.get_albums(), sparse_output=True).fit_transform(albums_list)
        icm_albums_csr = icm_albums.tocsr()
        #return icm_albums_csr
        
        #durations= self.tracks.reindex(columns=['track_id', 'duration_sec'])
        #durations.sort_values(by='track_id', inplace=True)
        #durations_list = [[d] for d in durations['duration_sec']]
        #icm_durations = MultiLabelBinarizer(classes=self.get_durations(), sparse_output=True).fit_transform(durations_list)
        #icm_durations_csr= icm_durations.tocsr()
        
        return sparse.hstack((0*icm_artists_csr,0*icm_albums_csr))



In [100]:
b = Builder()
URM_train, URM_test = b.train_test_holdout( 0.8)

[ 1220  8360 12844 14301 18397]


In [154]:
recommender = ItemCBFKNNRecommender(URM_train, b.get_ICM())
contentSimilarity = recommender.fit(shrink=0.0, topK=500)

#recommender = ItemCBFKNNRecommender(URM_train, b.get_URM_transpose())
#collaborativeSimilarity = recommender.fit(shrink=5.0, topK=500)

Similarity column 20600 ( 100 % ), 2729.54 column/sec, elapsed time 0.13 min


In [73]:
from evaluation_function import evaluate_algorithm
from Compute_Similarity_Python import Compute_Similarity_Python
class ItemCBFKNNRecommender(object):
    
    def __init__(self, URM, ICM):
        self.URM = URM
        self.ICM = ICM
    
    
    def fit(self, topK=50, shrink=100, normalize = True, similarity = "cosine"):
        
        similarity_object = Compute_Similarity_Python(self.ICM.T, shrink=shrink,
        topK=topK, normalize=normalize,
        similarity = similarity)

        self.W_sparse = similarity_object.compute_similarity()

        #print (self.W_sparse)
        sparse.save_npz('cbsim.npz', self.W_sparse, compressed=True)
        sparse_matrix = sparse.load_npz('cbsim.npz')
        #print (sparse_matrix)
        return self.W_sparse
    
    
    def recommend(self, user_id, at=None, exclude_seen=True):
        # compute the scores using the dot product
        user_profile = self.URM[user_id]
        scores = user_profile.dot(self.W_sparse).toarray().ravel()
        
        if exclude_seen:
            scores = self.filter_seen(user_id, scores)
        
        # rank items
        ranking = scores.argsort()[::-1]
        
        return ranking[:at]
    
    
    def filter_seen(self, user_id, scores):
        
        start_pos = self.URM.indptr[user_id]
        end_pos = self.URM.indptr[user_id+1]
        
        user_profile = self.URM.indices[start_pos:end_pos]
        
        scores[user_profile] = -np.inf
        
        return scores


In [142]:
import random
class hybridRecommender():

    def merge(self, a, b):
        hybridMatrix = a*contentSimilarity + b*collaborativeSimilarity
        return hybridMatrix

    def __init__(self):
        self.bestSimilarTracks = self.merge(1, 0)
        #self.df = pd.read_csv('input/train.csv')
        #df1 = pd.read_csv('target_playlists.csv')
        #self.playlists = np.array(df1['playlist_id'])
        #submission = np.empty((len(playlists),2), dtype=object)
        self.cont = -1
    
    def recommend(self, playlist):
        self.cont = self.cont + 1

        tracks = b.get_tracks_inside_playlist_train(playlist)
        #submission[cont][0] = playlist
        tracksSet = set(tracks)
        best = {}
        temp = 1
        for track in tracks:
            row_start = self.bestSimilarTracks.indptr[track]
            row_end = self.bestSimilarTracks.indptr[track+1]
            similarTracks = self.bestSimilarTracks.indices[row_start:row_end]
            similarityValues = self.bestSimilarTracks.data[row_start:row_end]
            for i in range(0, len(similarTracks)):
                if not similarTracks[i] in tracksSet:
                    if similarTracks[i] in best:
                        best[similarTracks[i]]=best[similarTracks[i]]-similarityValues[i]*temp
                    else:
                        best[similarTracks[i]]=-1*similarityValues[i]*temp
            if self.cont < 5000:
                temp -= 1/(len(tracks)+1)
        preSorted = [[v, k] for k,v in best.items()]
        best = np.empty((max(11,len(preSorted)), 2), dtype=object)
        for i in range(len(preSorted)):
            best[i] = preSorted[i]
        if len(preSorted) < 11:
            for i in range(len(preSorted), 11):
                best[i] = [0, random.randint(0, 20000)]
            print(best)
        best = best[best[:,0].argpartition(10)][0:10]
        best = best[best[:,0].argsort()][:,1]
        return best

In [108]:
print(recommend(10))

NameError: name 'recommend' is not defined

In [115]:
import numpy as np
import scipy.sparse as sps



def precision(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score



def recall(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score



def MAP(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score



def evaluate_algorithm(URM_test, recommender_object, at=5):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0

    num_eval = 0

    URM_test = sps.csr_matrix(URM_test)

    n_users = URM_test.shape[0]

    ordered_target_playlists = b.get_ordered_target_playlists()
    unordered_target_playlists = b.get_unordered_target_playlists()

    for i in range(len(ordered_target_playlists)):
        
        user_id = ordered_target_playlists[i]

        start_pos = URM_test.indptr[user_id]
        end_pos = URM_test.indptr[user_id+1]

        if end_pos-start_pos>0:

            
            relevant_items = URM_test.indices[start_pos:end_pos]

            recommended_items = recommender_object.recommend(user_id)
            
            if i == 0:
                print(user_id)
                print(recommended_items)
                print(relevant_items)
            num_eval+=1

            is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

            cumulative_precision += precision(is_relevant, relevant_items)
            cumulative_recall += recall(is_relevant, relevant_items)
            cumulative_MAP += MAP(is_relevant, relevant_items)
            
    print("ordered finished")
        
    for i in range(len(unordered_target_playlists)):
        
        user_id = unordered_target_playlists[i]

        start_pos = URM_test.indptr[user_id]
        end_pos = URM_test.indptr[user_id+1]

        if end_pos-start_pos>0:

                
            relevant_items = URM_test.indices[start_pos:end_pos]

            recommended_items = recommender_object.recommend(user_id)
            
            if i == 0:
                print(user_id)
                print(recommended_items)
                print(relevant_items)
            num_eval+=1

            is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

            cumulative_precision += precision(is_relevant, relevant_items)
            cumulative_recall += recall(is_relevant, relevant_items)
            cumulative_MAP += MAP(is_relevant, relevant_items)


    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval

    print("Recommender performance is: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP))

    result_dict = {
        "precision": cumulative_precision,
        "recall": cumulative_recall,
        "MAP": cumulative_MAP,
    }

    return result_dict

In [155]:
evaluate_algorithm(URM_test, hybridRecommender(),at=10)

7
[734 4492 2273 13076 15124 15908 6099 6615 13158 19638]
[  159  1775  7660 12493 13424 15740 17495]
[[-3.384613770705004 5907]
 [-3.384613770705004 9405]
 [-3.7692289719214815 15328]
 [-3.384613770705004 19158]
 [-0.3846152012164777 1831]
 [-0.7692304024329554 2173]
 [-0.7692304024329554 4811]
 [-0.3846152012164777 11989]
 [-0.6923073621896602 9130]
 [-0.9999995231628424 9365]
 [0 16263]]
[[-0.4999997615814209 1089]
 [-0.28571414947509777 256]
 [-0.28571414947509777 15814]
 [-0.28571414947509777 18644]
 [-0.14285707473754894 3204]
 [-0.14285707473754894 9961]
 [-0.14285707473754894 10841]
 [-0.14285707473754894 14742]
 [-0.14285707473754894 16662]
 [0 14137]
 [0 17193]]
[[-4.714283466339111 1588]
 [-4.035712361335754 4850]
 [-3.7142839431762695 6426]
 [-4.428569316864014 8540]
 [-4.714283466339111 12668]
 [-4.714283466339111 13402]
 [-4.035712361335754 13624]
 [-4.035712361335754 15873]
 [-4.035712361335754 16905]
 [0 233]
 [0 3656]]
[[-0.20833323399225892 2996]
 [-0.2083332339922589

[[-0.44444423251681836 1459]
 [-0.33333317438761373 14552]
 [0 9025]
 [0 18778]
 [0 7447]
 [0 17620]
 [0 5465]
 [0 3121]
 [0 2725]
 [0 12530]
 [0 18902]]
[[-0.9374995529651642 248]
 [-0.9374995529651642 685]
 [-0.9374995529651642 4868]
 [-1.8749991059303284 6581]
 [-0.9374995529651642 18570]
 [-1.624999225139618 4925]
 [-1.124999463558197 5263]
 [-1.124999463558197 13160]
 [-1.124999463558197 17592]
 [0 9116]
 [0 6372]]
[[-3.7499982118606563 5801]
 [-4.33333126703898 11399]
 [-2.999998569488525 16155]
 [-0.7499996423721311 4848]
 [0 6841]
 [0 4373]
 [0 14561]
 [0 11728]
 [0 12466]
 [0 3878]
 [0 4942]]
[[-1.2222216394212508 5199]
 [-0.4999997615814207 1096]
 [-0.4999997615814207 4249]
 [-0.4999997615814207 10358]
 [-0.4999997615814207 12226]
 [-0.4999997615814207 14476]
 [-0.4999997615814207 15424]
 [-0.4999997615814207 15944]
 [-0.9999995231628414 18826]
 [0 236]
 [0 14670]]
[[-0.4999997615814209 13956]
 [0 5773]
 [0 1125]
 [0 5426]
 [0 13317]
 [0 15986]
 [0 9844]
 [0 12454]
 [0 5400]


[[-4.999997615814209 6717]
 [-1.4999992847442627 9738]
 [-4.999997615814209 11399]
 [-4.999997615814209 13115]
 [-4.499997854232788 13926]
 [-3.4999983310699463 16155]
 [-4.999997615814209 19935]
 [0 6693]
 [0 15977]
 [0 16017]
 [0 6619]]
[[-2.9999985694885254 5334]
 [-2.9999985694885254 9208]
 [-5.999997138977051 11817]
 [-2.9999985694885254 12968]
 [-5.999997138977051 14561]
 [-2.9999985694885254 18622]
 [-5.999997138977051 20230]
 [-5.999997138977051 20532]
 [-0.4999997615814209 10569]
 [0 16560]
 [0 18866]]
[[-2.9999985694885254 698]
 [-1.9999990463256836 2380]
 [-2.4999988079071045 5275]
 [-2.4999988079071045 5694]
 [-2.4999988079071045 9962]
 [-2.4999988079071045 14065]
 [-2.9999985694885254 18180]
 [-0.4999997615814209 1872]
 [-0.4999997615814209 4289]
 [-0.4999997615814209 19114]
 [0 10844]]
[[-4.499997854232788 5801]
 [-5.999997138977051 6717]
 [-3.999998092651367 9833]
 [-3.999998092651367 12322]
 [-3.4999983310699463 16155]
 [-4.499997854232788 19935]
 [-0.9999995231628418 9

{'precision': 0.03428000000000022,
 'recall': 0.08342834748584765,
 'MAP': 0.03876557780612233}